# 06 — Fine Tuning und Robustheit

## Ziel
Dieses Notebook beschreibt den Prozess der Feinabstimmung unseres Modells und die Bewertung seiner Robustheit gegenüber verschiedenen Störungen in den Eingabedaten.

## Imports und Setup

In [ ]:
import importlib
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score, davies_bouldin_score
import utils.paths as paths
from utils.data_loader import load_all
from utils.reporting import ensure_dirs, save_joblib, save_json
from utils.tasks.success_pct import SuccessPctTrainer

import utils.tasks.success_residual as success_residual
import utils.tasks.success_pct as success_pct
import utils.tasks.hit as hit
import utils.tasks.moods as moods
import utils.tasks.ranker as ranker
import utils.tasks.artist_trajectory as artist_trajectory
import utils.tasks.artist_clustering as artist_clustering
import utils.tasks.track_similarity as track_similarity

# reload modules
importlib.reload(success_pct)
importlib.reload(success_residual)
importlib.reload(hit)
importlib.reload(moods)
importlib.reload(ranker)
importlib.reload(artist_trajectory)
importlib.reload(artist_clustering)
importlib.reload(track_similarity)

# use classes from the module (always up to date)
SuccessPctTrainer = success_pct.SuccessPctTrainer
SuccessResidualTrainer = success_residual.SuccessResidualTrainer
HitTrainer = hit.HitTrainer
MoodTrainer = moods.MoodTrainer
RankerTrainer = ranker.RankerTrainer
ArtistTrajectoryTrainer = artist_trajectory.ArtistTrajectoryTrainer
ArtistClusteringRunner = artist_clustering.ArtistClusteringRunner
TrackSimilarityRunner = track_similarity.TrackSimilarityRunner

from utils.datasets import  (
    build_success_pct_dataset,
    build_success_residual_dataset,
    build_hit_dataset,
    build_mood_dataset,
)

from utils.config import RANDOM_SEED, ALLOW_LEAKY_FEATURES

importlib.reload(paths)

SAMPLE_NAME = paths.load_sample_name()
PATHS = paths.make_paths(SAMPLE_NAME)
paths.ensure_dirs(PATHS)



## Load Data

In [ ]:
data = load_all(PATHS.input_targets_path)

ds_pct  = build_success_pct_dataset(data.X_track, data.track_df, data.y_success_pct)
ds_res  = build_success_residual_dataset(data.X_track, data.track_df, data.y_success_residual)
ds_hit  = build_hit_dataset(data.X_track, data.track_df, data.y_hit)
ds_mood = build_mood_dataset(data.X_track, data.track_df, data.Y_mood)

ap = data.artist_panel.sort_values("release_month_ts").reset_index(drop=True)

y_artist_growth = ap["y_growth"].astype(float)
y_artist_breakout = ap["y_breakout"].astype(int)

X_artist_panel = ap.select_dtypes(include=["number", "bool"]).drop(
    columns=["y_growth", "y_breakout", "release_month_ts", "artist_id"],
    errors="ignore"
).fillna(0)



## Tuning: Success Percentile innerhalb der Kohorte (Regression)

Ziel: Vorhersage der relativen Erfolgsposition eines Tracks innerhalb seiner Release-Kohorte (0–100).

Optimierungsziel:
- **MAE auf dem Validierungsset minimieren**, da die Skala interpretierbar ist (Punkte im Perzentilraum).

Hinweis:
- Wir nutzen einen **kohortenbasierten Zeitsplit**, um Leakage in die Zukunft zu vermeiden.
- Optional kann XGBoost auf **GPU (CUDA)** laufen (`device="cuda"`).


In [ ]:
pct_trainer = SuccessPctTrainer(seed=RANDOM_SEED)

best_pct = pct_trainer.tune(
    ds_pct,
    n_trials=40,
    device="gpu",   # oder "cpu"
)

best_pct

## Tuning: Success Residual innerhalb der Kohorte (Regression)

Ziel: Modellierung von **Über- oder Unterperformance** relativ zur Kohorte (Residual statt absoluter Erfolg).

Optimierungsziel:
- **MAE auf dem Validierungsset minimieren** (robust, gut interpretierbar).

Hinweis:
- Residual-Targets sind oft verrauscht → Regularisierung ist besonders wichtig.
- Kohortenbasierter Zeitsplit verhindert ungewollte Zukunftsinformation.


In [ ]:
res_trainer = SuccessResidualTrainer(seed=RANDOM_SEED)

best_res = res_trainer.tune(
    ds_res,
    n_trials=50,
    device="cuda",   # oder "cpu"
)

best_res



## Tuning: Hit Prediction (Binary Classification)

Ziel: Vorhersage, ob ein Track als „Hit“ gilt.

Optimierungsziel:
- **PR-AUC auf dem Validierungsset maximieren**
  (bei unausgeglichenen Klassen oft sinnvoller als ROC-AUC).

Zusatz:
- Der finale **Threshold** (für F1) wird später auf der Validierung optimiert (wie im Training-Notebook).
- GPU ist möglich (XGBoost).


In [ ]:
hit_trainer = HitTrainer(seed=RANDOM_SEED)

best_hit = hit_trainer.tune(
    ds_hit,
    n_trials=60,
    device="cuda",   # oder "cpu"
)

best_hit


## Tuning: Mood Prediction (Multi-Label)

Ziel: Mehrere Mood-Labels pro Track (z.B. happy, sad, chill …).

Optimierungsziel:
- **Micro-F1 auf dem Validierungsset maximieren**
  (gute Standardmetrik bei Multi-Label, weil sie alle Entscheidungen gemeinsam bewertet).

Hinweis:
- Viele sklearn-Modelle laufen CPU-basiert; GPU ist hier meistens nicht relevant.
- Thresholds können später pro Label separat optimiert werden.


In [ ]:
mood_trainer = MoodTrainer(seed=RANDOM_SEED)

best_mood = mood_trainer.tune(
    ds_mood,
    n_trials=30,
)

best_mood


## Tuning: Ranker (Learning-to-Rank)

Ziel: Tracks innerhalb einer Kohorte sinnvoll zu ranken (Top-K Qualität).

Optimierungsziel:
- **Mean NDCG@10 auf dem Validierungsset maximieren**.

Wichtig:
- Ranking benötigt eine saubere Gruppierung (z.B. pro `cohort_ym`).
- GPU ist möglich (XGBRanker).


In [ ]:
rank_trainer = RankerTrainer(seed=RANDOM_SEED)

best_rank = rank_trainer.tune(
    ds_pct,        # typischerweise nutzt der Ranker das gleiche X, aber anderes Training/Grouping
    n_trials=30,
    device="cuda", # oder "cpu"
    k=10
)

best_rank


## Tuning: Artist Trajectory (Growth & Breakout)

Ziel: Künstlerentwicklung über Zeit modellieren.

Teilaufgaben:
- **Growth (Regression):** log1p-transformiertes Wachstum → **MAE minimieren**
- **Breakout (Binary):** Breakout-Event → **PR-AUC maximieren**

Hinweis:
- Zeitliche Sortierung ist Pflicht, damit der Split korrekt ist.
- GPU ist möglich (XGBoost).


In [ ]:
artist_trainer = ArtistTrajectoryTrainer(seed=RANDOM_SEED)

best_artist = artist_trainer.tune(
    data.artist_panel,
    n_trials=30,
    device="cuda",   # oder "cpu"
)

best_artist


##  Unsupervised Tuning: Artist Clustering (KMeans)

In diesem Schritt optimieren wir das Clustering **ohne Ground Truth**.
Dafür testen wir verschiedene Parameter (z.B. `k` und PCA-Dimensionen) auf einem **Sample**, um die Laufzeit gering zu halten.

**Bewertung (Heuristiken):**
- **Silhouette Score** (höher = besser getrennte Cluster)
- **Davies–Bouldin Index** (niedriger = kompakter / besser)

Anschließend trainieren wir das beste Setup auf dem vollständigen Datensatz und erzeugen optional eine 2D-PCA-Visualisierung.


In [ ]:
RNG = np.random.RandomState(RANDOM_SEED)

# Sample Artists für schnellere Evaluierung
sample_n = 8000
idx = RNG.choice(len(data.artist_df), size=sample_n, replace=False)
artist_sample = data.artist_df.iloc[idx].copy()

def eval_clustering(k, pca_dim=16, scale=True):
    runner = ArtistClusteringRunner(k=k, seed=RANDOM_SEED, scale=scale, pca_dim=pca_dim)
    models, artifact, extra = runner.run(artist_sample)

    X_used = extra["X_used"]
    labels = extra["labels"]

    sil = silhouette_score(X_used, labels)
    db = davies_bouldin_score(X_used, labels)

    return {
        "k": k,
        "pca_dim": pca_dim,
        "scale": scale,
        "silhouette": float(sil),
        "davies_bouldin": float(db),
        "artifact": artifact,
    }

candidates = []
for k in [10, 15, 20, 25, 30, 35, 40, 50]:
    for pca_dim in [8, 12, 16]:
        candidates.append(eval_clustering(k=k, pca_dim=pca_dim, scale=True))

df_c = pd.DataFrame([{k: v for k, v in c.items() if k != "artifact"} for c in candidates])
df_c.sort_values(["silhouette", "davies_bouldin"], ascending=[False, True]).head(10)


In [ ]:
best_row = df_c.sort_values(["silhouette", "davies_bouldin"], ascending=[False, True]).iloc[0]
best_cluster_cfg = best_row.to_dict()
best_cluster_cfg


In [ ]:
cluster_runner = ArtistClusteringRunner(
    k=int(best_cluster_cfg["k"]),
    seed=RANDOM_SEED,
    scale=bool(best_cluster_cfg["scale"]),
    pca_dim=int(best_cluster_cfg["pca_dim"]),
)
cluster_models, cluster_artifact, cluster_extra = cluster_runner.run(data.artist_df)

# optional plot on sample
cluster_runner.plot_pca2(cluster_extra["X_used"], cluster_extra["labels"])


##  Embeddings & Track Similarity: Plausibilitäts-Checks

Track-Similarity ist in der Praxis oft **qualitativ** zu bewerten (klingen/fühlen sich die Nachbarn ähnlich an?).
Hier führen wir daher einen stabilen Check durch:

- Embedding einmal fitten
- Mehrere **gültige Track-Keys** auswählen
- Für jeden Key die Top-K ähnlichsten Tracks abrufen

Ziel: Sicherstellen, dass die Pipeline funktioniert, die Keys korrekt gematcht werden und die Ergebnisse plausibel sind.


In [ ]:
sim_runner = TrackSimilarityRunner()
sim_models, sim_artifact = sim_runner.fit(data.track_df)

# choose valid keys correctly (avoid the '0' string issue)
track_keys = data.track_df.index.to_numpy()

RNG = np.random.RandomState(RANDOM_SEED)
example_keys = RNG.choice(track_keys, size=5, replace=False)

examples = {}
for key in example_keys:
    examples[str(key)] = sim_runner.get_similar(
        track_key=key,
        track_index=data.track_df.index,
        embeddings=sim_models["embeddings"],
        top_k=10
    )

examples


## Finalisierung: Best-Modelle retrainen & Artefakte speichern

Jetzt retrainen wir jedes Modell mit den **besten gefundenen Hyperparametern** und speichern:

- Modelle nach `PATHS.tuned_models_dir`
- Tuning-Report (Scores + Parameter + Artefakte) nach `PATHS.reports_dir_tuned`

Damit ist die Tuning-Runde reproduzierbar und die Ergebnisse sind sauber versioniert.


In [ ]:
tuned_report = {}
# success_pct
pct_model, pct_metrics = pct_trainer.fit_eval(ds_pct, params=best_pct["best_params"])
save_joblib({"model": pct_model, "best_params": best_pct}, paths.tuned_models_dir / "success_pct_tuned.joblib")
tuned_report["success_pct"] = {"best": best_pct, "metrics": pct_metrics}

# success_residual
res_model, res_metrics = res_trainer.fit_eval(ds_res, params=best_res["best_params"])
save_joblib({"model": res_model, "best_params": best_res}, paths.tuned_models_dir / "success_residual_tuned.joblib")
tuned_report["success_residual"] = {"best": best_res, "metrics": res_metrics}

# hit
hit_model, hit_metrics, hit_thr = hit_trainer.fit_eval(ds_hit, params=best_hit["best_params"])
save_joblib({"model": hit_model, "threshold": hit_thr, "best_params": best_hit}, paths.tuned_models_dir / "hit_tuned.joblib")
tuned_report["hit"] = {"best": best_hit, "metrics": hit_metrics}

# mood (optional)
if best_mood is not None:
    mood_model, mood_metrics, mood_thresholds = mood_trainer.fit_eval(ds_mood, params=best_mood["best_params"])
    save_joblib({"model": mood_model, "thresholds": mood_thresholds, "best_params": best_mood}, paths.tuned_models_dir / "mood_tuned.joblib")
    tuned_report["mood"] = {"best": best_mood, "metrics": mood_metrics}

# ranker (optional)
if "best_rank" in globals() and best_rank is not None:
    rank_model, rank_metrics = rank_trainer.fit_eval(ds_pct, params=best_rank["best_params"])
    save_joblib({"model": rank_model, "best_params": best_rank}, paths.tuned_models_dir / "ranker_tuned.joblib")
    tuned_report["ranker"] = {"best": best_rank, "metrics": rank_metrics}

# artist clustering (save fitted full model)
save_joblib({"models": cluster_models, "artifact": cluster_artifact, "best_cfg": best_cluster_cfg},
            paths.tuned_models_dir / "artist_clustering_tuned.joblib")
tuned_report["artist_clustering"] = {"best_cfg": best_cluster_cfg, "artifact": cluster_artifact}

# track similarity (save embedding)
save_joblib({"models": sim_models, "artifact": sim_artifact}, paths.tuned_models_dir / "track_similarity.joblib")
tuned_report["track_similarity"] = sim_artifact

# final tuned report json
tuned_report["run_config"] = {"seed": RANDOM_SEED, "allow_leaky_features": ALLOW_LEAKY_FEATURES}
save_json(tuned_report, paths.reports_dir_tuned / "tuned_metrics_report.json")

tuned_report.keys()
